## 第5週の3日目、さて...

### AutoGen Core

コレは少し変わったものです。

これは基盤となるエージェント・フレームワークに依存しません。

AutoGen AgentChat を使用することも、他のものを使用することもできます。

これはエージェント・インタラクション・フレームワークです。

その観点から見ると、LangGraph と似た位置付けになります。

### 基本原則

Autogen Core は、エージェントのロジックとメッセージの配信方法を分離します。

このフレームワークは通信インフラストラクチャとエージェントのライフサイクルを提供し、エージェントは自身の処理に責任を持ちます。

この通信インフラストラクチャはランタイムと呼ばれ、**スタンドアロン** と **分散** の 2 つのタイプがあります。

今日は、スタンドアロン・ランタイムである、ローカル埋め込みエージェント・ランタイム実装である **SingleThreadedAgentRuntime** を使用します。

明日は分散ランタイムについて簡単に見ていきます。

### 概要
「AutoGen Core」というエージェント対話フレームワークの概要と基本的な使い方を説明する。

主な内容は以下の通りだが、最終段階の目的をみると、ココでは、  
`RoutedAgent`を使った階層型のマルチエージェントの構築を目標にしている模様。

- AutoGen Coreの紹介
  - AutoGen Coreはエージェントのロジックとメッセージ配信の仕組みを分離するフレームワーク
  - ランタイムという通信基盤を提供し、エージェントは自身の仕事に集中できる。
  - Standalone（単体）とDistributed（分散）の2種類のランタイムがある。
  - 本NBではStandalone型（`SingleThreadedAgentRuntime`）を使う。

- メッセージ・オブジェクトの定義  
エージェント間でやりとりするメッセージ（`Message`クラス）をシンプルに定義

- エージェントの実装例  
`RoutedAgent`を継承した`SimpleAgent`を定義、特定のメッセージを受け取ると定型文で返答

- Runtimeの起動とメッセージ送信  
エージェントをRuntimeに登録し、メッセージ送信・応答を実行

- LLM（大規模言語モデル）エージェントの導入  
OpenAIやOllamaなどのLLM APIを利用するエージェント（`MyLLMAgent`など）を定義し、より高度な応答を生成。

- 階層型のマルチエージェントの対話例
  - 3体のエージェント（2人のプレイヤーと審判役）で「じゃんけん」を行う
  - プレイヤー役はそれぞれ異なるLLMバックエンドを利用し、審判役が結果判定

In [1]:
# import

from dotenv import load_dotenv
from dataclasses import dataclass

from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_ext.models.openai import OpenAIChatCompletionClient
#from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

# 初期化
load_dotenv(override=True)

True

### まず、Message オブジェクトを定義します。

Agent フレームワークでメッセージに使用したい構造です。

In [2]:
# シンプルなものにしましょう!

@dataclass
class Message:
    content: str


### エージェントを定義します

RoutedAgent のサブクラスです。

- すべてのエージェントには **Agent ID** があり、これは 2 つの要素から構成されます。
 - `agent.id.type` はエージェントの種類を表します。
 - `agent.id.key` はエージェントの一意の識別子です。

- `@message_handler` でデコレートされたメソッドはすべて、メッセージを受信できます。

...そもそも、RoutedAgentとは、ルーティング専用のエージェントで、  
入力されたタスクやメッセージを解析し、適切なエージェントに 振り分ける（routeする） のが仕事。

In [3]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        #return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        return Message(content=f"これは {self.id.type}-{self.id.key} です。あなたは「{message.content}」と言いましたが、私は同意しません。")
        

### スタンドアロン・ランタイムを作成し、エージェント・タイプを登録

In [4]:
# 先の、スタンドアロン・ランタイムである、ローカル埋め込みエージェント・ランタイム実装
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

### よし！ランタイムを起動してメッセージを送信しよう。

In [5]:
# 開始
runtime.start()

In [6]:
# simple_agent の id を取得、id を指定してメッセージを送信
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("やあ、こんにちは！"), agent_id)
print(">>>", response.content)

>>> これは simple_agent-default です。あなたは「やあ、こんにちは！」と言いましたが、私は同意しません。


In [7]:
# 停止
await runtime.stop()
await runtime.close()

### OK では、もっと面白いことをしてみましょう。

AgentChat アシスタントを使います！

In [8]:
# 本来のRoutedAgent（ルーティング専用のエージェント）はタスクを解析して他のエージェントへ振り分ける

# RoutedAgentクラス定義
class MyLLMAgent(RoutedAgent):

    # 初期化
    def __init__(self) -> None:
        # エージェントの識別名
        super().__init__("LLMAgent")
        
        # 振り分け先のエージェント
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    # メッセージハンドラの定義
    @message_handler # メッセージ受信のデコレータ
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        
        # 受け取ったメッセージをログに出力
        print(f"{self.id.type} received message: {message.content}")
        
        # 受け取った Message を、LLM用の TextMessage に変換
        text_message = TextMessage(content=message.content, source="user")
        
        # on_messages: AssistantAgent に LLM 呼び出しを委譲
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)

        # LLM から返ってきた応答を取り出し
        reply = response.chat_message.content

        # ログ出力
        print(f"{self.id.type} responded: {reply}")

        # Messageとして返却
        return Message(content=reply) 

In [9]:
# 先の、スタンドアロン・ランタイムである、ローカル埋め込みエージェント・ランタイム実装
runtime = SingleThreadedAgentRuntime()

# 2つのRoutedAgent（ルーティング専用のエージェント）を登録
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [10]:
# 開始
runtime.start()

LLMAgent received message: こんにちは！
LLMAgent responded: こんにちは！今日はどんなことをお手伝いしましょうか？
LLMAgent received message: これは simple_agent-default です。あなたは「こんにちは！今日はどんなことをお手伝いしましょうか？」と言いましたが、私は同意しません。
LLMAgent responded: ご意見ありがとうございます。どのようにお手伝いできるか、具体的なリクエストがあれば教えてください。


In [11]:
# ユーザー の prompt → LLMAgent
response = await runtime.send_message(Message("こんにちは！"), AgentId("LLMAgent", "default"))
print(">>>", response.content)

# LLMAgent の response → simple_agent
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)

# simple_agent → LLMAgent
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))
print(">>>", response.content)

>>> こんにちは！今日はどんなことをお手伝いしましょうか？
>>> これは simple_agent-default です。あなたは「こんにちは！今日はどんなことをお手伝いしましょうか？」と言いましたが、私は同意しません。
>>> ご意見ありがとうございます。どのようにお手伝いできるか、具体的なリクエストがあれば教えてください。


In [12]:
# 停止
await runtime.stop()
await runtime.close()

### さて、実際にこれが動作するか見てみましょう
3つのエージェントが対話するようにしましょう。

In [13]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [14]:
# OllamaChatCompletionClient上げるのメンドイので...
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0) # OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [15]:
JUDGE = "あなたはジャンケンの試合を判定しています。プレイヤーたちは以下の手を出しました。:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        
        instruction = "あなたはじゃんけんをしています。次のうちのいずれか一つの単語だけで答えてください：グー、パー、チョキ。"

        # ※ インスタンスを生成するのではなく取得する。
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")        
        
        # メッセージを２つのプレイヤーに送る
        message = Message(content=instruction)
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)

        # 結果
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        
        # 判定
        judgement = f"{JUDGE}{result} 誰が勝ちましたか？"
        message = TextMessage(content=judgement, source="user")

        # 結果判定結果
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)

In [16]:
# 先の、スタンドアロン・ランタイムである、ローカル埋め込みエージェント・ランタイム実装
runtime = SingleThreadedAgentRuntime()

# 3つのRoutedAgent（ルーティング専用のエージェント）を登録
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))

AgentType(type='rock_paper_scissors')

In [17]:
# 開始
runtime.start()

In [18]:
# JUDGEに指示
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: グー
Player 2: グー
ジャンケンでは、同じ手を出した場合は引き分けとなります。したがって、Player 1とPlayer 2の双方がグーを出したため、勝者はありません。引き分けです。 

TERMINATE


In [19]:
# 停止
await runtime.stop()
await runtime.close()